# **User guide**

This notebook presents the procedure to follow in order to generate and use the descriptors defined in HLA-EpiCheck. This procedure could be used to process new antigen data (new MD trajectories) or to reproduce the results obtained in the paper. It is assumed that each antigen has a directory and that this directory contains the DCD and PSF files of the MD trajectory of each antigen.

## **Run MD simulations**

1. Get an initial structure for each antigen to process (from PDB or Alphafold).

2. Generate the psf file of each structure using VMD.

3. Solvate and ionize each structure using VMD.

4. Modify the relevant fields/parameters in the NAMD configuration files 'general_min.conf' and 'general_MD.conf' (see the data folder).

5. Run the minimization and MD simulation using NAMD.

6. Clean the MD traejctories (remote waters and keep the relevant frames).


## **Generation of the descriptors**

1. Prepare a directory called "PDBs" inside each antigen's directory. This directory will contain the set of PDB files that represent the MD trajectory of the antigen.

2. Compute SASA and RSASA values using the script 'RSASA_trajectory.py':

    USAGE : python RSASA_trajectory.py <data_directory>

    <data_directory> corresponds to the absolute path to  the antigen directories which contain the data to be processed.

    Run the script from the same directory where it is located.

    Attention! This script requires the python package 'VMD-PYTHON'.

3. Compute RMSF values using the script 'run_batch_RMSF.sh':

    USAGE: ./run_batch_RMSF.sh  <number of frames> <data_directory> 
    
    <number of frames> corresponds to the number of frames to process.
    
    <data_directory> corresponds to the absolute path to the antigen directories which contain the data to be processed.
    
    Run the script from the same directory where it is located.
    
    This script will generate the RMSF data for each antigen on files like this : <antigen name>_RMSF.dat

4. Compute the pre-patches using the script 'compute_prepatches.tcl'. We recommend using 'gnu parallel' to distribute the calculations across several nodes (command used in a OAR cluster):

    Run in a bash shell: 

    ```
    dir_data=<data_directory>

    nohup parallel --joblog $PWD/parallel.log --progress -j 60 --ssh oarsh --sshloginfile $OAR_NODEFILE vmd -dispdev text -e $PWD/compute_patches.tcl -args $dir_data/{1}/PDBs 15  ::: $(while read line; do antigen=${line%/} && antigen=${antigen##*/} && echo $antigen; done < $dir_data/list_antigens.txt) > vmd_patches_out.log 2> vmd_patches_err.log &

    ```

    <data_directory> corresponds to the absolute path to the antigen directories which contain the data to be processed.
    
    For more information about 'compute_prepatches.tcl', please see the comments in the script.


5. Run the 'dataset_gen_radius_15.ipynb' notebook:

    Please pay attention to the required packages in the 'Imports' section.

    The variables 'radius_patch' and 'thresh_surf' in the  'Definitions' section set the patch size and the solven-accessibility threshold parameters respectively. 

    Please modify the 'dir_data' variable in the 'Definitions' section. This variable define the absolute path to the antigen directories.

# **Training (just for reproducibility purpose, not necessary for making predictions)**

## **Applying the patch filter (removing patches according to the inclusion criteria described in the article)**

Please modify the 'dir_data' variable in the first cell of this section. This variable define the absolute path to the antigen directories.

In [ ]:
import pandas as pd
import os
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

def convert_name(name):
    if name.split('_')[0] in ['A', 'B', 'C']:
        new_name = f"{name.split('_')[0]}*{name.split('_')[1][:2]}:{name.split('_')[1][2:4]}"

    elif (len(name.split('_')[0]) > 2) and (name.split('_')[0][:3] == 'DQB'):
        new_name = f"DQB1*{name.split('_')[1][:2]}:{name.split('_')[1][2:4]}-" + \
                    f"DQA1*{name.split('_')[2][:2]}:{name.split('_')[2][2:4]}"

    elif (len(name.split('_')[0]) > 2) and (name.split('_')[0][:3] == 'DRB'):
        new_name = f"{name.split('_')[0]}*{name.split('_')[1][:2]}:{name.split('_')[1][2:4]}"

    elif name.split('_')[0] in ['DP', 'DQ']:
        new_name = f"{name.split('_')[0]}B1*{name.split('_')[1][5:7]}:{name.split('_')[1][7:9]}-" + \
                    f"{name.split('_')[0]}A1*{name.split('_')[1][:2]}:{name.split('_')[1][2:4]}"
    return new_name

def formal2informal_allele(name):
    if name.split('*')[0] in ['A', 'B', 'C']:
        groupe = name.split('*')[0]
        new_name = f"{groupe}_{name.split('*')[1][:2]}{name.split(':')[1][:2]}"
    elif name.split('*')[0][:2] in ['DP', 'DQ']:
        groupe = name.split('*')[0][:2]
        new_name = f"{name.split('*')[1][:2]}{name.split(':')[1][:2]}"
    elif name.split('*')[0][:2] in ['DR']:
        groupe = name.split('*')[0]
        new_name = f"{groupe}_{name.split('*')[1][:2]}{name.split(':')[1][:2]}"
    return new_name

def formal2informal_antigen(name):
    if name.split('*')[0] in ['A', 'B', 'C']:
        groupe = name.split('*')[0]
        new_name = f"{groupe}_{name.split('*')[1][:2]}{name.split(':')[1][:2]}"
    elif name.split('*')[0][:2] in ['DP', 'DQ']:
        groupe = name.split('*')[0][:2]
        new_name = f"{groupe}_{name.split('-')[1][5:7]}{name.split('-')[1][8:10]}-{name.split('*')[1][:2]}{name.split('*')[1][3:5]}"
    elif name.split('*')[0][:2] in ['DR']:
        groupe = name.split('*')[0]
        new_name = f"{groupe}_{name.split('*')[1][:2]}{name.split(':')[1][:2]}"
    return new_name

dic_hydro_KD = {
                'G': -0.4,
                'A': 1.8,
                'V': 4.2,
                'F': 2.8,
                'P': -1.6,
                'M': 1.9,
                'I': 4.5,
                'L': 3.8,
                'D': -3.5,
                'E': -3.5,
                'K': -3.9,
                'R': -4.5,
                'S': -0.8,
                'T': -0.7,
                'Y': -1.3,
                'H': -3.2,
                'C': 2.5,
                'N': -3.5,
                'Q': -3.5,
                'W': -0.9
}

dic_hydro_E = {
                'G': 0.48,
                'A': 0.62,
                'V': 1.08,
                'F': 1.19,
                'P': 0.12,
                'M': 0.64,
                'I': 1.38,
                'L': 1.06,
                'D': -0.9,
                'E': -0.74,
                'K': -1.5,
                'R': -2.53,
                'S': -0.18,
                'T': -0.05,
                'Y': 0.26,
                'H': -0.4,
                'C': 0.29,
                'N': -0.78,
                'Q': -0.85,
                'W': 0.81
}

dic_charge = {
                'G': 0,
                'A': 0,
                'V': 0,
                'F': 0,
                'P': 0,
                'M': 0,
                'I': 0,
                'L': 0,
                'D': -1,
                'E': -1,
                'K': 1,
                'R': 1,
                'S': 0,
                'T': 0,
                'Y': 0,
                'H': 0,  
                'C': 0,
                'N': 0,
                'Q': 0,
                'W': 0, 
}


dic_groupe_lens = {'A':375, 'B':375, 'C':375, 'DP':374, 'DQ':378, 'DR':374}
dic_locus_lens = {'A':276, 'B':276, 'C':276, 'B2':99, 'DQA':186, 'DQB':192, 'DRA':182, 'DRB':192,
                   'DPA':183, 'DPB':191}

base_dir = os.getcwd()
pdb2fasta = f'{base_dir}/pdb2fasta.sh' 
surf_meth = 'RSASA'
hydro_scale = 'KD'

dir_data='/home/damaya/capsid_new/HLA-EpiCheck'

radius_patch = 15
thresh_surf = 20

list_avail_alleles = []
with open(f'{dir_data}/list_antigens.txt', 'r') as f:
    for line in f:
        list_avail_alleles.append(line.strip().split('/')[-2])

In [ ]:
table_3_5 = pd.read_csv(f'{dir_data}/table_3_{surf_meth}_{thresh_surf}_hydro_{hydro_scale}_radius_{radius_patch}.csv', sep=',', index_col=0)

dir_eplets = f'{base_dir}/../data'
df_all_eplets = pd.read_csv(f'{dir_eplets}/lists_All_confirmed_eplets.csv', index_col=0)
df_all_no_conf_eplets = pd.read_csv(f'{dir_eplets}/lists_All_non_confirmed_eplets.csv', index_col=0)
df_eplets_ext = pd.DataFrame(columns=['list_resids'])
df_no_conf_eplets_ext = pd.DataFrame(columns=['list_resids'])
track_resids_patchs = pd.read_csv(f'{dir_data}/track_resids_patchs_table_0_{surf_meth}_{thresh_surf}_hydro_{hydro_scale}_radius_{radius_patch}.csv', sep=',', index_col=0)
table_2 = pd.read_csv(f'{dir_data}/table_2_{surf_meth}_{thresh_surf}_hydro_{hydro_scale}_radius_{radius_patch}.csv', sep=',', index_col=0)

for allele in list_avail_alleles:
    if allele.split('_')[0] in ['DP', 'DQ']:
        allele = allele.strip()
        locus_A = allele.split('_')[0] if allele.split('_')[0] in ['A', 'B', 'C'] else allele.split('_')[0][:2]+'A'
        locus_B = 'B2' if allele.split('_')[0] in ['A', 'B', 'C'] else allele.split('_')[0][:2]+'B'
                
        name_A = allele.split('_')[0]+'A1*'+allele.split('-')[0][3:5]+':'+allele.split('-')[0][5:7]
        name_B = allele.split('_')[0]+'B1*'+allele.split('-')[1][0:2]+':'+allele.split('-')[1][2:4]
        eplets_A = df_all_eplets[df_all_eplets['allele_name'] == name_A]
        eplets_B = df_all_eplets[df_all_eplets['allele_name'] == name_B]
        df_eplets_ext.loc[name_A, 'list_resids'] = set() 
        for index, row in eplets_A.iterrows():
            list_conf_eplets = list(filter(None, eplets_A.loc[index, 'eplet_resid_nums'].split(' ')))
            df_eplets_ext.loc[name_A, 'list_resids'] = df_eplets_ext.loc[name_A, 'list_resids'].union(set(map(int, list_conf_eplets)))
        
        df_eplets_ext.loc[name_B, 'list_resids'] = set()
        for index, row in eplets_B.iterrows():
            list_conf_eplets = list(filter(None, eplets_B.loc[index, 'eplet_resid_nums'].split(' ')))
            df_eplets_ext.loc[name_B, 'list_resids'] = df_eplets_ext.loc[name_B, 'list_resids'].union(set(map(int, list_conf_eplets)))
        
        no_conf_eplets_A = df_all_no_conf_eplets[df_all_no_conf_eplets['allele_name'] == name_A]
        no_conf_eplets_B = df_all_no_conf_eplets[df_all_no_conf_eplets['allele_name'] == name_B]
        
        df_no_conf_eplets_ext.loc[name_A, 'list_resids'] = set()
        for index, row in no_conf_eplets_A.iterrows():
            list_no_conf_eplets = list(filter(None, no_conf_eplets_A.loc[index, 'eplet_resid_nums'].split(' ')))
            df_no_conf_eplets_ext.loc[name_A, 'list_resids'] = df_no_conf_eplets_ext.loc[name_A, 'list_resids'].union(set(map(int, list_no_conf_eplets)))
            
        df_no_conf_eplets_ext.loc[name_B, 'list_resids'] = set()
        for index, row in no_conf_eplets_B.iterrows():
            list_no_conf_eplets = list(filter(None, no_conf_eplets_B.loc[index, 'eplet_resid_nums'].split(' ')))
            df_no_conf_eplets_ext.loc[name_B, 'list_resids'] = df_no_conf_eplets_ext.loc[name_B, 'list_resids'].union(set(map(int, list_no_conf_eplets)))

    else:
        allele = allele.strip()
        locus_A = allele.split('_')[0] if allele.split('_')[0] in ['A', 'B', 'C'] else allele.split('_')[0][:2]+'A'
        locus_B = 'B2' if allele.split('_')[0] in ['A', 'B', 'C'] else allele.split('_')[0][:2]+'B'
                
        name_A = allele.split('_')[0]+'*'+allele.split('_')[1][0:2]+':'+allele.split('_')[1][2:4]
        eplets_A = df_all_eplets[df_all_eplets['allele_name'] == name_A]
        df_eplets_ext.loc[name_A, 'list_resids'] = set()
        for index, row in eplets_A.iterrows():
            df_eplets_ext.loc[name_A, 'list_resids'] = df_eplets_ext.loc[name_A, 'list_resids'].union(set(map(int, eplets_A.loc[index, 'eplet_resid_nums'].split(' '))))
        
        no_conf_eplets_A = df_all_no_conf_eplets[df_all_no_conf_eplets['allele_name'] == name_A]
        df_no_conf_eplets_ext.loc[name_A, 'list_resids'] = set()
        
        for index, row in no_conf_eplets_A.iterrows():
            list_no_conf_eplets = list(filter(None, no_conf_eplets_A.loc[index, 'eplet_resid_nums'].split(' ')))
            df_no_conf_eplets_ext.loc[name_A, 'list_resids'] = df_no_conf_eplets_ext.loc[name_A, 'list_resids'].union(set(map(int, list_no_conf_eplets)))

df_seqs = pd.DataFrame()
for locus in ['A', 'B', 'C', 'DPA1', 'DPB1', 'DQA1', 'DQB1', 'DRA', 'DRB']:
    df = pd.read_csv(f'{base_dir}/../data/{locus}_nr_results.txt', sep='\t', header=None)
    df_seqs = pd.concat([df_seqs, df], axis=0, ignore_index=True)

for index, col in df_seqs.iterrows():
    antigen = df_seqs.at[index, 0].split('|')[0]
    locus = antigen.split('*')[0].strip()
    antigen = f'{locus}_{antigen.split(":")[0].split("*")[1]}{antigen.split(":")[1]}'
    df_seqs.at[index, 0] =  antigen

conserved_pos = {'A':[], 'B':[], 'C':[], 'DP':[], 'DQ':[], 'DRB':[]}
dic_eplet_cons = {'A':set(), 'B':set(), 'C':set(), 'DP':set(), 'DQ':set(), 'DRB':set()}
distri_test = []   
for locus in ['A', 'B', 'C', 'DP', 'DQ', 'DRB']:
    if locus in ['A', 'B', 'C']:
        antigens_simu = [x for x in list_avail_alleles if locus == x.split('_')[0]]
        
        for pos in range(dic_locus_lens[locus]):
            var_antigens = set()
            polym = set()
            for index, col in df_seqs[df_seqs[0].isin(antigens_simu)].iterrows():
                polym.add(df_seqs.at[index, 1][pos])
                var_antigens.add(df_seqs.at[index, 0])
                
            if len(polym) == 1: conserved_pos[locus].append(pos+1)
        conserved_pos[locus] = conserved_pos[locus] + [x for x in range(277,376)]
        
        
    if locus in ['DP', 'DQ']:
        antigens_simu = [x for x in list_avail_alleles if locus == x.split('_')[0]]
        antigens_simu_A = [f'{locus}A1_{x.split("_")[1][:4]}' for x in antigens_simu]
        antigens_simu_B = [f'{locus}B1_{x.split("-")[1]}' for x in antigens_simu]

        for pos in range(dic_locus_lens[locus+'A']):
            var_antigens = set()
            polym = set()
            for index, col in df_seqs[df_seqs[0].isin(antigens_simu_A)].iterrows():
                polym.add(df_seqs.at[index, 1][pos])
                var_antigens.add(df_seqs.at[index, 0])
                
            if len(polym) == 1: conserved_pos[locus].append(pos+1)
        
        for pos in range(dic_locus_lens[locus+'B']):
            var_antigens = set()
            polym = set()
            for index, col in df_seqs[df_seqs[0].isin(antigens_simu_B)].iterrows():
                polym.add(df_seqs.at[index, 1][pos])
                var_antigens.add(df_seqs.at[index, 0])
                
            if len(polym) == 1: conserved_pos[locus].append(dic_locus_lens[locus+'A'] \
                                                            + pos + 1)
        
    if locus in ['DRB']:
        antigens_simu = [x for x in list_avail_alleles if locus in x.split('_')[0]]
        
        for pos in range(dic_locus_lens['DRB']):
            var_antigens = set()
            polym = set()
            for index, col in df_seqs[df_seqs[0].isin(antigens_simu)].iterrows():
                polym.add(df_seqs.at[index, 1][pos])
                var_antigens.add(df_seqs.at[index, 0])
                
            if len(polym) == 1: conserved_pos[locus].append(dic_locus_lens['DRA'] \
                                                            + pos + 1)
        conserved_pos[locus] = conserved_pos[locus] + [x for x in range(1,183)]
            
for index, col in table_3_5[table_3_5['class'] == 0].iterrows():
    allele = track_resids_patchs[track_resids_patchs['patch_ID'] == \
                                  col['patch_ID']]['antigen'].iloc[0]
    central_AA = track_resids_patchs[track_resids_patchs['patch_ID'] == \
                                  col['patch_ID']]['central_AA'].iloc[0]
    patch_ID = col['patch_ID']
    col_names = table_2[table_2['patch_ID'] == patch_ID].T.dropna().index
    resids_patch = {int(name.split('_')[-1]) for name in col_names if 'N_RMSF_resid' in name}
    
    if allele.split('_')[0] in ['DP', 'DQ']:
        allele = allele.strip()
        name_A = allele.split('_')[0]+'A1*'+allele.split('-')[0][3:5]+':'+allele.split('-')[0][5:7]
        name_B = allele.split('_')[0]+'B1*'+allele.split('-')[1][0:2]+':'+allele.split('-')[1][2:4]
        eplets_A_ext = df_eplets_ext.loc[name_A, 'list_resids']
        eplets_B_ext = df_eplets_ext.loc[name_B, 'list_resids']
        eplets_B_ext = set(map(lambda x:x+dic_locus_lens[allele.split('_')[0]+'A'], eplets_B_ext))
        non_conf_eplets_A = df_no_conf_eplets_ext.loc[name_A, 'list_resids']
        non_conf_eplets_B = df_no_conf_eplets_ext.loc[name_B, 'list_resids']
        non_conf_eplets_B = set(map(lambda x:x+dic_locus_lens[allele.split('_')[0]+'A'], \
                                    non_conf_eplets_B))
        conf_eplets = eplets_A_ext.union(eplets_B_ext)
        non_conf_eplets = non_conf_eplets_A.union(non_conf_eplets_B)
        eplets_allele = conf_eplets.union(non_conf_eplets)
        
    if allele.split('_')[0] in ['A', 'B', 'C']:
        allele = allele.strip()
        name_A = allele.split('_')[0]+'*'+allele.split('_')[1][0:2]+':'+allele.split('_')[1][2:4]
        eplets_A_ext = df_eplets_ext.loc[name_A, 'list_resids']
        non_conf_eplets_A = df_no_conf_eplets_ext.loc[name_A, 'list_resids']
        eplets_allele = eplets_A_ext.union(non_conf_eplets_A)
        
    if allele.split('_')[0][:2] in 'DR':
        allele = allele.strip()
        name_B = allele.split('_')[0]+'*'+allele.split('_')[1][0:2]+':'+allele.split('_')[1][2:4]
        eplets_B_ext = df_eplets_ext.loc[name_B, 'list_resids']
        eplets_B_ext = set(map(lambda x:x+dic_locus_lens['DRA'], eplets_B_ext))
        non_conf_eplets_B = df_no_conf_eplets_ext.loc[name_B, 'list_resids']
        non_conf_eplets_B = set(map(lambda x:x+dic_locus_lens['DRA'], \
                                    non_conf_eplets_B))
        eplets_allele = eplets_B_ext.union(non_conf_eplets_B)
    
    resids_patch.add(central_AA)
    groupe = allele.split('_')[0] if len(allele.split('_')[0]) < 3 else 'DRB'
    distri_test.append(len(resids_patch))
    dic_eplet_cons[groupe] = dic_eplet_cons[groupe].union(eplets_allele.intersection(set(conserved_pos[groupe])))
    if (len(resids_patch.intersection(eplets_allele)) != 0) or \
        (len(resids_patch.intersection(set(conserved_pos[groupe]))) == len(resids_patch)) :
        table_3_5.drop(index, axis=0, inplace=True)

table_3_5.to_csv(f'{dir_data}/table_3_5_{surf_meth}_{thresh_surf}_hydro_{hydro_scale}_radius_{radius_patch}.csv', index=False)


## **Redundancy reduction (90% identity)**

In [ ]:
df_ag = pd.DataFrame(columns=['patch_ID', 'antigen',  'class'])

for ix, col in table_3_5.iterrows():
    antigen = track_resids_patchs[track_resids_patchs['patch_ID'] == col['patch_ID']]['antigen'].iloc[0]
    df_ag.loc[len(df_ag)] = [col['patch_ID'], convert_name(antigen), col['class']]

df_clusters = pd.read_csv(f'{base_dir}/../data/DB_clu_0.9_all.tsv', sep='\t', index_col=None, names=['representative', 'member'])
dic_non_epitope = {i:(0, '') for i in df_clusters['representative'].unique()}
dic_clusters = {i:[] for i in df_clusters['representative'].unique()}
dic_clusters_antigens = {i:[] for i in df_clusters['representative'].unique()}
df_ag2 = df_ag.groupby(['antigen', 'class'])['antigen'].agg(['count'])
trace = []
for repre in df_clusters['representative'].unique():
    df_members = df_clusters[df_clusters['representative'] == repre]
    for member in df_members['member'].unique():
        dic_clusters[repre].append(member)
        if (len(member.split('*')[0]) > 1 ) and (member.split('*')[0][:2] in ['DQ',  'DP']):
            list_antigens = [i for i in df_ag['antigen'].unique() if member in i]
            for antigen in list_antigens:
                dic_clusters_antigens[repre].append(antigen)
                n_non_eplet = df_ag2.loc[(antigen, 0)].iloc[0] if (antigen, 0) in df_ag2.index else 0
                cond_1 = repre in ['DPB1*10:01'] and antigen in ['DPB1*28:01-DPA1*01:05']
                cond_2 = repre in ['DQB1*03:03'] and antigen in ['DQB1*02:01-DQA1*05:08']
                cond_3 = repre in ['DQB1*03:02'] and antigen in ['DQB1*06:09-DQA1*01:02']
                cond_4 = repre in ['DPB1*15:01'] and antigen in ['DPB1*28:01-DPA1*01:05']
                cond_5 = repre in ['DQA1*01:02'] and antigen in ['DQB1*06:09-DQA1*01:02']
                if not (cond_1 or cond_2 or  cond_3 or cond_4 or cond_5):
                    if dic_non_epitope[repre][0] < n_non_eplet:
                        dic_non_epitope[repre] = (n_non_eplet, antigen)

        else: 
            dic_clusters_antigens[repre].append(member)
            n_non_eplet = df_ag2.loc[(member, 0)].iloc[0] if (member, 0) in df_ag2.index else 0
            if dic_non_epitope[repre][0] < n_non_eplet:
                dic_non_epitope[repre] = (n_non_eplet, member)


df_all_eplets = pd.read_csv(f'{base_dir}/../data/All_confirmed_eplets.csv', index_col=None, usecols=[ 'eplet_name', 'eplet_residues', 'allele_name'])

df_nr_dataset = pd.DataFrame(columns=['locus', 'eplet_name', 'resid', 'antigen', 'patch_ID'])
for cluster in dic_non_epitope.keys():
    if (len(cluster.split('*')[0]) > 1 ) and (cluster.split('*')[0][:2] in ['DQ',  'DP']):
        locus = cluster.split('*')[0][:2]
        list_alleles = dic_non_epitope[cluster][1].split('-')

        for allele in list_alleles:
            df = df_all_eplets[df_all_eplets['allele_name'] == allele]
        
            for ix, col in df.iterrows():
                list_resids = [i for i in df.loc[ix, 'eplet_residues'].split('(') if not ')' in i]
                list_resids = list_resids[0].strip().split(' ')
                list_resid_num = []
                for resid in list_resids:
                    resid_num = ''
                    for i in resid:
                        if i.isdigit():
                            resid_num += i
                        else:
                            resid_name = i
                            break
                    resid_num = int(resid_num) if locus+'A' in allele.split('*')[0] else int(resid_num)+dic_locus_lens[locus+'A']
                    patch = track_resids_patchs.loc[(track_resids_patchs['antigen'] == formal2informal_antigen(dic_non_epitope[cluster][1])) & \
                            (track_resids_patchs['central_AA'] == resid_num), 'patch_ID']
                    patch = patch.iloc[0] if len(patch) > 0 else float('nan')
                    df_nr_dataset.loc[len(df_nr_dataset)] = [locus, df.loc[ix, 'eplet_name'], resid_num,
                                                            dic_non_epitope[cluster][1], patch]

    else:
        locus = cluster.split('*')[0][:2] if len(cluster.split('*')[0]) > 1 else cluster.split('*')[0]
        df = df_all_eplets[df_all_eplets['allele_name'] == dic_non_epitope[cluster][1]]
        
        for ix, col in df.iterrows():
            list_resids = [i for i in df.loc[ix, 'eplet_residues'].split('(') if not ')' in i]
            list_resids = list_resids[0].strip().split(' ')
            list_resid_num = []
            for resid in list_resids:
                resid_num = ''
                for i in resid:
                    if i.isdigit():
                        resid_num += i
                    else:
                        resid_name = i
                        break
                resid_num = int(resid_num) if locus in ['A', 'B', 'C'] else int(resid_num)+dic_locus_lens['DRA']
                antigen = dic_non_epitope[cluster][1]
                patch = track_resids_patchs.loc[(track_resids_patchs['antigen'] == formal2informal_allele(antigen)) & \
                        (track_resids_patchs['central_AA'] == int(resid_num)), 'patch_ID']
                patch = patch.iloc[0] if len(patch) > 0 else float('nan')
                df_nr_dataset.loc[len(df_nr_dataset)] = [locus, df.loc[ix, 'eplet_name'], int(resid_num),
                                                        antigen, patch]

df_nr_dataset = df_nr_dataset.dropna()
df_nr_dataset.index = range(len(df_nr_dataset))

dic_epitopes_clusters = {i:[] for i in dic_clusters.keys()}

for key in dic_clusters_antigens.keys():
    for antigen in dic_clusters_antigens[key]:
        if not antigen in dic_non_epitope[key][1]:
            dic_epitopes_clusters[key].append(formal2informal_antigen(antigen))

for cluster in dic_epitopes_clusters.keys():
    if (len(cluster.split('*')[0]) > 1 ) and (cluster.split('*')[0][:2] in ['DQ',  'DP']):
        locus = cluster.split('*')[0][:2]
        for antigen in dic_epitopes_clusters[cluster]:
            
            list_alleles = antigen.split('-')
            list_alleles = [f'{locus}A1*{list_alleles[0][3:5]}:{list_alleles[0][5:]}',
                           f'{locus}B1*{list_alleles[1][:2]}:{list_alleles[1][2:]}']

            for allele in list_alleles:
                df = df_all_eplets[df_all_eplets['allele_name'] == allele]
                list_eplets = list(df['eplet_name'].unique())
                
                for eplet in list_eplets:
                    if eplet not in df_nr_dataset['eplet_name'].unique():
                        row = df[df['eplet_name'] == eplet]
                        list_resids = [i for i in row['eplet_residues'].iloc[0].split('(') if not ')' in i]
                        list_resids = list_resids[0].strip().split(' ')
                        list_resid_num = []
                        
                        for resid in list_resids:
                            resid_num = ''
                            for i in resid:
                                if i.isdigit():
                                    resid_num += i
                                else:
                                    resid_name = i
                                    break
                            resid_num = int(resid_num) if locus+'A' in allele.split('*')[0] else int(resid_num)+dic_locus_lens[locus+'A']
                            patch = track_resids_patchs.loc[(track_resids_patchs['antigen'] == antigen) & \
                                    (track_resids_patchs['central_AA'] == resid_num), 'patch_ID']
                            patch = patch.iloc[0] if len(patch) > 0 else float('nan')
                            df_nr_dataset.loc[len(df_nr_dataset)] = [locus, eplet, resid_num, convert_name(antigen), patch]

    else:
        locus = cluster.split('*')[0][:2] if len(cluster.split('*')[0]) > 1 else cluster.split('*')[0]
        for antigen in dic_epitopes_clusters[cluster]:
            df = df_all_eplets[df_all_eplets['allele_name'] == convert_name(antigen)]
            list_eplets = list(df['eplet_name'].unique())
            for eplet in list_eplets:
                if eplet not in df_nr_dataset['eplet_name'].unique():
                    row = df[df['eplet_name'] == eplet]
                    list_resids = [i for i in row['eplet_residues'].iloc[0].split('(') if not ')' in i]
                    list_resids = list_resids[0].strip().split(' ')
                    list_resid_num = []
                    for resid in list_resids:
                        resid_num = ''
                        for i in resid:
                            if i.isdigit():
                                resid_num += i
                            else:
                                resid_name = i
                                break
                        resid_num = int(resid_num) if locus in ['A', 'B', 'C'] else int(resid_num)+dic_locus_lens['DRA']
                        patch = track_resids_patchs.loc[(track_resids_patchs['antigen'] == antigen) & \
                                (track_resids_patchs['central_AA'] == int(resid_num)), 'patch_ID']
                        patch = patch.iloc[0] if len(patch) > 0 else float('nan')
                        df_nr_dataset.loc[len(df_nr_dataset)] = [locus, eplet, int(resid_num), convert_name(antigen), patch]
            
df_nr_dataset = df_nr_dataset.dropna()
df_nr_dataset.index = range(len(df_nr_dataset))
df_nr_dataset['class'] = [1 for i in range(len(df_nr_dataset))]


for cluster in dic_non_epitope.keys():
    locus = cluster.split('*')[0][:2] if len(cluster.split('*')[0]) > 1 else cluster.split('*')[0]
    antigen = dic_non_epitope[cluster][1]
    df_non_eplets = df_ag[(df_ag['antigen'] == antigen) & (df_ag['class'] == 0)]
    for ix, col in df_non_eplets.iterrows():
        resid = track_resids_patchs.loc[track_resids_patchs['patch_ID'] == col['patch_ID'], 'central_AA'].iloc[0]
        df_nr_dataset.loc[len(df_nr_dataset)] = [locus, 'non-eplet', resid, antigen, col['patch_ID'], 0]

df_nr_dataset['patch_ID'] = list(map(int, df_nr_dataset['patch_ID']))

table_3_new = table_3_5[table_3_5['patch_ID'].isin(df_nr_dataset['patch_ID'])]


## **Training a ML model**

The learning algorithm can be changed. For example, you could try: RandomForestClassifier, GradientBoostingClassifier or DecisionTreeClassifier. Modify the learning parameters accordingly.


In [ ]:
dataset = table_3_new.sample(frac=1,random_state=42)

X = dataset.iloc[:, 1:-1]
y = dataset.iloc[:, -1]

model = ExtraTreesClassifier(criterion='gini', max_features=None, min_samples_leaf=1, 
                            min_samples_split=3, n_estimators=300, warm_start=False, n_jobs=-1, 
                             random_state=42)

model.fit(X, y)

# **Predictions on descriptors**

Here we show how to load and use the previously trained models mentioned in the paper and which are located in the 'ML_models' folder.

The variable 'preds' contains the predicted labels for each of the descriptors passed to the model. The label 0 corresponds to a "Non-epitope" prediction, while the label 1 corresponds to an "Epitope" prediction. The variable 'proba_preds' contains the prediction probabilities for each label of each descriptor, where the first element corresponds to the probability of label 0 and the second element to label 1.

In the example below, the patch descriptors of all non-confirmed eplets in the Eplet Registry are used. These descriptors are found in the file named "descriptors_non_confirmed_eplets.csv" located in the 'data' folder.

In [13]:
import pickle
import pandas as pd


with open(f'../ML_models/model_ExtraTrees_non-redundant_fulldataset.pkl', 'rb') as file:
    model = pickle.load(file)

descriptors = pd.read_csv(f'../data/descriptors_non_confirmed_eplets.csv', sep=',', index_col=0)

preds = model.predict(descriptors)
proba_preds = model.predict_proba(descriptors)